In [1]:
import torch

In [2]:
dataset_file = './dataset/motionx_seq_text.zip'
!unzip -o -d ./data ./dataset/motionx_seq_text.zip

unzip:  cannot find either ./dataset/motionx_seq_text.zip or ./dataset/motionx_seq_text.zip.zip.


In [3]:
!unzip -o -d ./data ./dataset/motionx_smplx.zip

unzip:  cannot find either ./dataset/motionx_smplx.zip or ./dataset/motionx_smplx.zip.zip.


In [4]:
import numpy as np
import pandas as pd
motion = np.load('datasets/motion_data/smplx_322/HAA500/subset_0001/Badminton_Underswing.npy')
motion = torch.tensor(motion).float()
semantic_text = pd.read_csv('datasets/texts/semantic_labels/HAA500/subset_0001/Badminton_Underswing.txt')
semantic_text

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/motion_data/smplx_322/HAA500/subset_0001/Badminton_Underswing.npy'